# __getattr__ 

This illustrates when the **__getattr__** method is called

In [4]:
class LazyDB(object):
    
    def __init__(self):
        self.exists = 5
        
    def __getattr__(self, name):
        print("Called __getattr__ in LazyDB (%s)" %name)
        value = "Value for %s" %name
        setattr(self, name, value)
        return value

In [26]:
data = LazyDB()
print("Before:", data.__dict__)

Before: {'exists': 5}


In [27]:
print("Adding an atribute: ", data.foo)

Called __getattr__ in LazyDB (foo)
Adding an atribute:  Value for foo


In [28]:
print("After:", data.__dict__)

After: {'exists': 5, 'foo': 'Value for foo'}


In [29]:
data.baa

Called __getattr__ in LazyDB (baa)


'Value for baa'

In [30]:
print(data.__dict__)

{'exists': 5, 'foo': 'Value for foo', 'baa': 'Value for baa'}


In [31]:
data.baa

'Value for baa'

In [32]:
print(data.__dict__)

{'exists': 5, 'foo': 'Value for foo', 'baa': 'Value for baa'}


How to retrieve value of attributes using string name

In [36]:
print(getattr(data, 'exists'))

5


In [37]:
print(getattr(data, 'foo'))

Value for foo


### when does __getattr__ is called 

In [5]:
class LoggingLazyDB(LazyDB):
    
    def __getattr__(self, name):
        print("Called __getattr__(%s)" %name)
        return super().__getattr__(name)

In [9]:
data = LoggingLazyDB()
print('Exists:', data.exists)
print('foo: ', data.foo)
print('foo: ', data.foo)

Exists: 5
Called __getattr__(foo)
foo:  Value for foo
foo:  Value for foo


This show that **\_\_getattr__** is only reached when a variable is defined for the **the first time**

The *exists* attributes is present from the start, so **\_\_getattr__** is never called for it.

Because *foo* is not in the instance dictionary, so **\_\_getattr__** is called.

### \_\_getattribute__ 

This method is called every time an attribute is accessed on an object (even when it does already exist)

In [71]:
class ValidatingDB(object):
    
    def __init__(self):
        self.exists = 5
        
    def __getattribute__(self, name):
        print('Called __getattribute__ (%s)' %name)
        try:
            print('super() called worked on %s' %name)
            return super().__getattribute__(name)
        except AttributeError:
            #if variable does not exist yet, setattr is called
            value = "Value for %s" %name
            setattr(self, name, value)
            return value

In [67]:
data = ValidatingDB()

Called __getattribute__ (__class__)
super() called worked on __class__
Called __getattribute__ (__class__)
super() called worked on __class__


In [68]:
print('exists:', data.exists)

Called __getattribute__ (exists)
super() called worked on exists
exists: 5


In [69]:
print('foo: ', data.foo)

Called __getattribute__ (foo)
super() called worked on foo
foo:  Value for foo


In [70]:
print('foo: ', data.foo)

Called __getattribute__ (foo)
super() called worked on foo
foo:  Value for foo


### Check for missing attribute 

In [72]:
class MissingPropertyDB(object):
    
    def __init__(self):
        self.exists = 10
    
    def __getattr__(self, name):
        if name == 'bad_name':
            raise AttributeError("%s is missing in __getattr__ call" %name)
            
    def __getattribute__(self, name):
        if name == 'bad_name':
            raise AttributeError("%s is missing in __getattribute__ call" %name)

In [73]:
data = MissingPropertyDB()

Called __getattribute__ (__class__)
super() called worked on __class__
Called __getattribute__ (__class__)
super() called worked on __class__


In [80]:
data.bad_name

Called __getattr__(bad_name)


'Value for bad_name'

### \_\_hasattr\_\_ and \_\_getattr\_\_

### example 1 

In [7]:
data = LoggingLazyDB()
print("checking the class:")
print(hasattr(LoggingLazyDB, 'foo'))
print('Before: ', data.__dict__)
print('foo exists? ', hasattr(data, 'foo'))
print('After: ', data.__dict__)
print('foo exists? ', hasattr(data, 'foo'))

checking the class:
False
Before:  {'exists': 5}
Called __getattr__(foo)
Called __getattr__ in LazyDB (foo)
foo exists?  True
After:  {'foo': 'Value for foo', 'exists': 5}
foo exists?  True


When **hasattr** is called on an instance, it creates the instance attributes by calling **\_\_getattr__**,
also created when **getattr** is called !

In [79]:
print(data.__dict__)
print('Before: ', getattr(data, 'baa'))
print(data.__dict__)

{'exists': 5, 'foo': 'Value for foo'}
Called __getattr__(baa)
Before: %s Value for baa
{'exists': 5, 'foo': 'Value for foo', 'baa': 'Value for baa'}


### example 2

In [1]:
class ObjectCreator(object):
    pass

In [9]:
print(hasattr(ObjectCreator, 'new_attribute'))

False


In [11]:
ObjectCreator.new_attribute = 'foo'

In [12]:
print(hasattr(ObjectCreator, 'new_attribute'))

True


### \_\_setattr\_\_ 

In [82]:
class SavingDB(object):
    
    def __setattr__(self, name, value):
        print('Saving data to DB in SavingDB')
        super().__setattr__(name, value)

In [84]:
class LoggingSavingDB(SavingDB):
    
    def __setattr__(self, name, value):
        print('Called __setattr__(%s, %r)' %(name, value))
        super().__setattr__(name, value)

In [85]:
data = LoggingSavingDB()
print('Before: ', data.__dict__)
data.foo = 5
print('After: ', data.__dict__)
data.foo = 7
print('Finally: ', data.__dict__)

Before:  {}
Called __setattr__(foo, 5)
Saving data to DB in SavingDB
After:  {'foo': 5}
Called __setattr__(foo, 7)
Saving data to DB in SavingDB
Finally:  {'foo': 7}


**\_\_setattr\_\_** is called every time we are doing an assignment to an attribute

But the problem is that sometimes we do not want to call **getattribute** and/or **setattr**

__example__
    
    def __getattribute__(self, name):
        print('Called __getattribute__(%s)' % name)
        return self._data[name]
        
this will loop for ever as we call **getattribute**, inside **getattribute**, etc     
        


**The solution** called the *super().\_\_getattribute\_\_*

    def __getattribute__(self, name):
        data_dict = super().__getattribute__('data')
        return data_dict[name]

Same thing with **\_\_setattr\_\_** where we will need to use **super().\_\_setattr\_\_**